# Imports and setting up environment

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import csv
import os
import sys
import time
import logging
import re
import struct
from commons import *
from gan_arch import *
from datetime import datetime
import metuhelpers

In [ ]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)
else:
    print("No compatible GPUs found")


# Constants and globals

In [ ]:
EXPERIMENT_NAME = "debuggingExperiment"
DATETIME =  datetime.now().strftime("%Y%m%d-%H%M%S")
EXPERIMENT_NAME = "_".join([EXPERIMENT_NAME, DATETIME])

METU_RAW_PATH = '/qarr/studia/magister/datasets/METU/930k_logo_v3/'
METU_DATASET_PATH = '/home/zenfur/magister/resized_930k_logo/'
EVAL_ORIGIN_PATH = '/qarr/studia/magister/datasets/METU/query_reversed/'
EVAL_DATASET_PATH = '/home/zenfur/magister/metu_eval_256sq/'
LOG_DIR = "siamese_logs/" 
MODEL_DIR = "model_checkpoints/"
LAST_MODEL = "/home/zenfur/magister/jupyter/siamese_model20210322-032225_1560/"
ANALYSIS_DIR = "plots/"

LOAD = False
TESTING = False
HIDE_WARNINGS = True

AUTOTUNE = tf.data.experimental.AUTOTUNE

BATCH_SIZE = 8
TRUE_BATCH_SIZE = BATCH_SIZE * 3
lastEpoch=1

# lastSample = tf.Variable(tf.zeros([24, 224, 224, 3], dtype=tf.dtypes.float64))
# lastModel = [tf.Variable(tf.zeros([25088,2], dtype=tf.dtypes.float32)), tf.Variable(tf.zeros([2], dtype=tf.dtypes.float32))]

In [ ]:
# tf.debugging.experimental.enable_dump_debug_info(LOG_DIR, tensor_debug_mode="FULL_HEALTH", circular_buffer_size=-1)

In [ ]:
def set_experiment_name(name):
    global EXPERIMENT_NAME, DATETIME
    EXPERIMENT_NAME = name
    DATETIME = datetime.now().strftime("%Y%m%d-%H%M%S")
    EXPERIMENT_NAME = "_".join([EXPERIMENT_NAME, DATETIME])

# Functions declarations and definitions

### Definitions of pipeline building functions

In [ ]:
@tf.function
def tf_get_filename(path):
    return tf.strings.regex_replace(path, "[^/]*/", "")


#@tf.function
def tf_read_image(path):
    # Retrieving the group number from file name
    img = tf.io.read_file(path)
    return tf.image.decode_jpeg(img, channels=3, dct_method='INTEGER_ACCURATE')


def tf_get_class_from_name(path):
    filename = tf_get_filename(path)
    group_number = tf.strings.to_number(
        tf.strings.regex_replace(filename, "-.*$", ""), 
        out_type=tf.dtypes.int32
    )
    return group_number


#@tf.function
def tf_convert_and_normalize_img(img):
    c = tf.constant(256.0, dtype=tf.dtypes.float32)
    img = tf.cast(img, tf.dtypes.float32)
    return tf.math.divide(img, c)


In [ ]:
def triplet_getter(reference_source):
    def get_triplet_by_index(triplet):
        # @triplet: tuple/tensor of indices in the images table
        return tf.gather(reference_source, triplet)
    return get_triplet_by_index


def translator_getter(translationTable):
    def translate_indices(triplet):
        return tf.gather(translationTable, triplet)
    return translate_indices

Definitions of data augmentation functions

In [ ]:
def get_center_slice(image):
    return tf.image.central_crop(image, 224/256)


def random_slice_224x224(image, seed):
    return tf.image.stateless_random_crop(image, [224,224,3], seed)


def random_rotate(image, seed):
    if seed > 3:
        seed = 0
    return tf.image.rot90(image, k=seed)


def random_color_shift(image, seed):
    cov = np.cov(tf.reshape(image, (3, -1)))
    eigvals, eigvecs = np.linalg.eig(cov)
    random_direction = tf.random.stateless_normal((3,), seed, mean=0, stddev=0.1, dtype=tf.dtypes.float32)
    correction = tf.matmul(tf.cast(eigvecs, dtype=tf.dtypes.float32), 
                           tf.reshape(random_direction*eigvals, (3,1)))
    return image + tf.tile(tf.reshape(correction, shape=(1,1,3)), (224,224,1))


def static_augment(image, seeds):
    sditer = iter(seeds)
    
    image = random_slice_224x224(image, next(sditer))
    image = random_rotate(image, next(sditer))
#         image = random_color_shift(image, next(sditer))
#     try:
#     except StopIteration:
#         pass
    
    return image
    
    
def augmenter_getter(rng):
    def augment(image):
        seeds = [rng.make_seeds(2)[0], rng.uniform([], minval=0, maxval=5, dtype=tf.int32)]
        return static_augment(image, seeds)
    return augment


### Custom triples sampler based on linear congruent generator 
The generator has period equal NumberOfUnlikeSamples*(NumberOfAlikeSamples-1)

In [ ]:
def triplet_generator(start, length, avoid, n, initial, prime=756212269):
    """
    results in sequence of triplets (<start:start+length>, <avoid>, <0:n-1 excluding start:start+length-1>)
    """
    start = np.array(start, dtype=np.int32)
    length = np.array(length, dtype=np.int32)
    avoid = np.array(avoid, dtype=np.int32)
    initial = np.array(initial, dtype=np.int32)
    current = initial
    unlikeCount = (n-length)
    length = length - 1
    modulo_base = length * unlikeCount
    multiplier = modulo_base*11*3+1
    while True:
        # Generating next modulo from sequence
        current = (multiplier * current + prime) % modulo_base
        like = (current % length)
        unlike = current // length
        
        # Calculating proper indices from random modulos
        like += start
        like += (like >= avoid)
        like = np.expand_dims(like, axis=1)
        
        unlike = unlike + ((unlike >= start) * (length+1))
        unlike = np.expand_dims(unlike, axis=1)
        
        for triplet in np.concatenate((like, np.expand_dims(avoid, axis=1), unlike), axis=1):
            yield triplet

def advance_triplet_generator(steps, length, n, initial, prime=756212269):
    length = np.array(length, dtype=np.int32)
    initial = np.array(initial, dtype=np.int32)
    steps = steps // n
    current = initial
    unlikeCount = (n-length)
    length = length - 1
    modulo_base = length * unlikeCount
    multiplier = modulo_base*11*3+1
    for i in range(steps):
        current = (multiplier * current + prime) % modulo_base
    return current

Helper functions

In [ ]:
def plot_results(selected_images, representations, scale=0.5, samples_per_image=5):
    """
    Function for drawing sheet of 
    """
    samples_per_image += 1
    fig, plots = plt.subplots(len(selected_images), 
                              samples_per_image, 
                              figsize=(80/12*scale/6*samples_per_image,130*len(selected_images)/120*scale)
                             )
    for i, selected in enumerate(selected_images):
        img = representations[selected]
        dist = lambda x: tf.sqrt(tf.reduce_sum((x - img)**2))
        reprs_distance = tf.map_fn(dist, representations)
        closest_idx = np.argsort(reprs_distance)
        for j, p in enumerate(closest_idx[0:samples_per_image]):
            plots[i][j].imshow(imagesTable[p].numpy())
            if j == 0:
                plots[i][j].set_title(f"{p}", fontsize=7)
            else:
                plots[i][j].set_title(f"{reprs_distance[p]:2.3}", fontsize=7)
            plots[i][j].axes.get_xaxis().set_visible(False)
            plots[i][j].axes.get_yaxis().set_visible(False)
            
    return fig, plots

### Defining custom tensorboard callback for logging and saving training progress

In [ ]:
class MeanTBCallback(tf.keras.callbacks.TensorBoard):
    def __init__(self, name="", previewInterval=0, drawSamplesList=None, *args, **kwargs):
        super(MeanTBCallback, self).__init__(*args, **kwargs)
        self.name = name or DATETIME
        self.previewInterval = previewInterval
        self.selectedToDraw = drawSamplesList or list()
        self.mean_train_loss = 0
        self.mean_test_loss = 0
        self.train_batches = 0
        self.test_batches = 0

    def on_epoch_end(self, epoch, logs=None):
        super(MeanTBCallback, self).on_epoch_end(epoch, logs=logs)
        global imagesTable
        # Tensorflow 2.2.0 breaks backwards compatibility here
        writer = self._train_writer
        
        if self.train_batches > 0:
            with self._train_writer.as_default():
                tf.summary.scalar("mean_loss", self.mean_train_loss/self.train_batches, step = epoch)

        if self.test_batches > 0:
            tf.print(f"Mean losses: train:{self.mean_train_loss/self.train_batches} val:{self.mean_test_loss/self.test_batches}")
            with self._val_writer.as_default():
                tf.summary.scalar("mean_loss", self.mean_test_loss/self.test_batches, step = epoch)

        # TODO split functionality into separate callbacks?
        if self.previewInterval > 0 and epoch % self.previewInterval == 0:
            start_time = time.time()

            # Saving the images plot
            tf.print(f"Saving sampling drawings {EXPERIMENT_NAME + '_' + str(epoch)}")
            reprs = self.model.predict(tf.image.central_crop(imagesTable, 224/256))
            norm_reps = self.model.normalize_output(reprs)
            f, p = plot_results(self.selectedToDraw, norm_reps, scale=1.0)
            f.savefig(f"{ANALYSIS_DIR}{EXPERIMENT_NAME}_{epoch}.png")
            del f
            del p
            plt.close()
            
            # Calculating ranks
            raw_rank = metuhelpers.calculate_rank(norm_reps, 
                                                  norm_reps, 
                                                  {k:[i for i in range(v[0], v[0]+v[1])] for k,v in imageGroups},
                                                  metuhelpers.euclidean_sq)

            ranks = np.mean([np.mean(x+0.5) for x in raw_rank.values() if x is not []])
            ranks_normalised = np.mean([(np.mean(x+0.5)-(len(x)+1)/2)/len(norm_reps) for x in raw_rank.values() if x is not []])

            # Logging the ranks
            self.log_value(ranks, "rank", epoch)
            self.log_value(ranks_normalised, "rank_normalised", epoch)
            tf.print(f"Rank {ranks:2.4f} Normalised: {ranks_normalised:2.4f}")

            end_time = time.time()
            tf.print(f'Time taken for evaluation and saving: {end_time-start_time} seconds')

        self.train_batches = 0
        self.test_batches = 0
        self.mean_train_loss = 0
        self.mean_test_loss = 0
        
        
    #@tf.function
    def on_train_batch_end(self, batch, logs=None):
        self.train_batches += 1
        if "loss" in logs:
            self.mean_train_loss += logs["loss"]

    #@tf.function
    def on_test_batch_end(self, batch, logs=None):
        self.test_batches += 1
        if "loss" in logs:
            self.mean_test_loss += logs["loss"]
            
    def log_value(self, value, value_name, epoch):
        with self._train_writer.as_default():
            tf.summary.scalar(value_name, value, step = epoch)



In [ ]:
class ModelSaverCallback(tf.keras.callbacks.Callback):
    def __init__(self, save_interval):
        self.save_interval = save_interval
        
    def on_epoch_end(self, epoch, logs=None):
        # Saving the model
        if self.save_interval > 0 and epoch % self.save_interval == 0:
            tf.print(f"Saving the model {EXPERIMENT_NAME + '_' + str(epoch)}")
            self.model.save(MODEL_DIR + EXPERIMENT_NAME + "_" + str(epoch))

### Siamese triplet loss function 

In [ ]:
def triplet_loss(alike, anchor, unlike, margin=1.0, reduce=tf.reduce_mean):
    a = tf.reduce_sum(tf.math.squared_difference(alike, anchor), axis=1)
    b = tf.reduce_sum(tf.math.squared_difference(unlike, anchor), axis=1)
    return reduce(tf.maximum(a + margin - b, 0.0))

# def triplet_loss(alike, anchor, unlike, margin=1.0, reduce=tf.reduce_mean):
#     a = tf.norm(alike-anchor, axis=1)
#     b = tf.norm(unlike-anchor, axis=1)
#     return reduce(tf.maximum(a + margin - b, 0.0))

### Defining custom triplet model class

In [ ]:
def normalize_output_with_max(x):
    return x / tf.expand_dims(tf.maximum(tf.math.reduce_max(x, axis=1), 1e-5), axis=1)

def normalize_output_with_max_batch(x):
    return x / tf.maximum(tf.math.reduce_max(x, axis=None), 1e-5)

def normalize_output_with_norm(x):
    return x / tf.expand_dims(tf.maximum(tf.norm(x, axis=1), 1e-5), axis=1)

In [ ]:
zero64 = tf.constant(0.0, dtype=tf.dtypes.float64)
zero32 = tf.constant(0.0, dtype=tf.dtypes.float32)
class TripletSiamese(tf.keras.models.Model):
    def __init__(self, shared_net, output_normalizer, name=None):
        super(TripletSiamese, self).__init__(name=name)
        self.siameseBase = shared_net
        self.normalize_output = output_normalizer
        self.callctr = 0

    def compile(self, optimizer, loss):
        super(TripletSiamese, self).compile()
        self.optimizer = optimizer
        self.loss = loss#lambda a,b,c: triplet_loss(a,b,c, margin=loss_margin)
    
    @tf.function#(jit_compile=True)
    def train_step(self, input_triplets):
#         h = 0
#         for l in np.ravel(input_triplets.numpy()):
#             h += hash(l)
#         tf.print(h)
        
    
#         global lastSample, lastModel
#         lastSample.assign(tf.add(input_triplets, zero64))
#         for i, layer in zip(self.siameseBase.trainable_weights, lastModel):
#             layer.assign(tf.add(i, zero32)) 
        with tf.GradientTape(persistent=True) as tape:
            # training and calculating the error function gradient
            representations = self.siameseBase(input_triplets, training=True)
            representations = self.normalize_output(representations)
            alike = tf.strided_slice(representations, [0,0], tf.shape(representations), strides=[3,1])
            anchor = tf.strided_slice(representations, [1,0], tf.shape(representations), strides=[3,1])
            unlike = tf.strided_slice(representations, [2,0], tf.shape(representations), strides=[3,1])
            loss = self.loss(alike, anchor, unlike)
        grads = tape.gradient(loss, self.siameseBase.trainable_weights)

        
#         alike_in = tf.strided_slice(input_triplets, [0,0,0,0], tf.shape(input_triplets), strides=[3,1,1,1])
#         anchor_in = tf.strided_slice(input_triplets, [1,0,0,0], tf.shape(input_triplets), strides=[3,1,1,1])
#         unlike_in = tf.strided_slice(input_triplets, [2,0,0,0], tf.shape(input_triplets), strides=[3,1,1,1])

#         with tf.GradientTape() as tape:
#             alike = self.siameseBase(alike_in, training=True)
#         grads1 = tape.gradient(alike, self.siameseBase.trainable_weights) 
        
#         with tf.GradientTape() as tape:
#             anchor = self.siameseBase(anchor_in, training=True)
#         grads2 = tape.gradient(anchor, self.siameseBase.trainable_weights) 
        
#         with tf.GradientTape() as tape:
#             unlike = self.siameseBase(unlike_in, training=True)
#         grads3 = tape.gradient(unlike, self.siameseBase.trainable_weights) 
        
#         tf.print(describe(grads1))
#         tf.print(describe(alike))
#         tf.print(describe(anchor))
#         grads1 *= 2*(alike - anchor)
#         grads2 *= 2*(unlike - alike)
#         grads3 *= 2*(alike - unlike)
        
#         grads = np.mean([grads1, grads2, grads3])# ...

#         loss = self.loss(alike, anchor, unlike)
#         h = 0
#         for l in tf.experimental.numpy.ravel(self.siameseBase.layers[-1].weights[0]):
#             h += hash(l)
#         tf.print("Przed ", h)
        
        self.optimizer.apply_gradients(zip(grads, self.siameseBase.trainable_weights))
        
#         tf.print(self.siameseBase.layers.trainable_weights)
#         h = 0
#         for l in tf.experimental.numpy.ravel(self.siameseBase.layers[-1].weights[0]):
#             h += hash(l)
#         tf.print("Po ", h)
        
        return {"loss": loss}
    
        
    def evaluate(self, x=None, y=None, batch_size=None, verbose=False, sample_weight=None, steps=None,
                callbacks=None, max_queue_size=10, workers=1, use_multiprocessing=False,
                return_dict=False):
        r = self.siameseBase.predict(x=x, batch_size=batch_size, verbose=False, steps=steps, callbacks=callbacks)
        r = self.normalize_output(r)
        alike = tf.strided_slice(r, [0,0], tf.shape(r), strides=[3,1])
        anchor = tf.strided_slice(r, [1,0], tf.shape(r), strides=[3,1])
        unlike = tf.strided_slice(r, [2,0], tf.shape(r), strides=[3,1])
        dct = {"loss":self.loss(alike, anchor, unlike, reduce=tf.reduce_mean)}
        if callbacks is not None:
            for cb in callbacks:
                cb.on_test_batch_end(x, logs=dct)
        if return_dict:
            return dct
        else:
            return dct["loss"]
    
    def save(self, path):
        self.siameseBase.save(path)
        
    def predict(self, *args, **kwargs):
        return self.siameseBase.predict(*args, **kwargs)

### Data analysis helper functions

In [ ]:
def getPCA(matrix):
    # matrix is 2D prediction data from neural net, first dimension is batch size, second is vector outputs
    # thus data is in columns
    cov = np.cov(matrix, rowvar=False)
    eigvals, eigvecs = np.linalg.eig(cov)
    return eigvals, eigvecs

# Pipeline initialisation

### Loading the dataset into RAM

In [ ]:
rngd = tf.random.Generator.from_seed(4563463)
def tf_dummy_read_image(path):
    return rngd.normal((256,256,3))

In [ ]:
# Loading a list of paths of dataset images
imagesList = tf.io.matching_files(EVAL_DATASET_PATH + "*.jpg")
DBlen = len(imagesList)

# Preparing dataset structure from the list of paths
evalpathsDB = tf.data.Dataset.from_tensor_slices(imagesList)

# Buidling the dataset pipeline of loading evaluation images for the purpose of loading them all into RAM
evalDB = (evalpathsDB.map(tf_read_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)
          .batch(32)
          .map(tf_convert_and_normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
)

# Buidling the dataset pipeline of image classifications
evalGrps = (evalpathsDB.map(tf_get_class_from_name, num_parallel_calls=tf.data.experimental.AUTOTUNE)
            .batch(32)
)

In [ ]:
labs = np.array(list(evalGrps.unbatch().as_numpy_iterator()))

np.random.seed(14200)

# Reordering the labels using numerical order, rather than alphabetical one
labsOrder = np.argsort(labs, kind="stable")
labs = labs[labsOrder]

Converting the database of samples into numpy array, since it can fit into RAM memory to save time.

In [ ]:
imagesTable = tf_db_to_array(evalDB, DBlen)
imagesTable = imagesTable[labsOrder]
imagesTable = tf.constant(imagesTable)

### Preparing summary of image classifications for further use
By convention, discarding images from group 0, as they have been manually inserted as the examples that differ from the rest sampled from 930k METU dataset.

Reordering the labels from alphabetical order into ascending by group number order.

In [ ]:
# Building the images classifications (groups) summary
imageGroupsDct = dict()
for i, l in enumerate(labs):
    last_count = imageGroupsDct.get(l,(i,0))
    imageGroupsDct[l] = (last_count[0], last_count[1] + 1)
# Removing the 0-th class, as it's a dummy class of non simliar images
del(imageGroupsDct[0])

### Splitting the dataset into validation and training subsets

Taking **validationUniques** samples from each class as uniqiue anchor samples in validation dataset. Pairing those with **xSamples** random samples from original class as similar (alike) sample and any other class as differing (unlike) sample.

In [ ]:
validationUniques = 2
validationSubset = [(a, [i for i in range(j[0],j[0] + validationUniques)]) for a,j in imageGroupsDct.items()]
validationSamples = []
xSamples = 40
N = len(labs)
# Setting the random seed for deterministic choice purposes - to make validation set exactly the same every time

for grp, samples in validationSubset:
    for sample in samples:
        groupStart =  imageGroupsDct[grp][0]
        groupLength = imageGroupsDct[grp][1]
        for i in range(xSamples):
            alike = np.random.randint(groupStart, groupStart + groupLength)
            unlike = np.random.randint(0, N-groupLength)
            unlike += (unlike>=groupStart)*groupLength
            validationSamples.append((alike, sample, unlike))

### Building the translation table

The table is necessary as removing some samples from the data set for validation purposes changes the ordering of remaining samples. Instead duplicating data separating those images, it was easier to make the translation on the fly into proper indices from remaining training samples.

In [ ]:
trainingTranslationTable = np.array(range(N-validationUniques*len(validationSubset)))
starts = [s[0] for s in imageGroupsDct.values()]
starts.sort()
starts.append(N+1)
j = 0
for i in range(len(trainingTranslationTable)):
    if starts[j] <= i+j*validationUniques:
        j += 1
    trainingTranslationTable[i] = i+j*validationUniques
trainingTranslationTable = tf.constant(trainingTranslationTable)

### Generating intialisation values for triplet generator

Generating the necessary initialisation values such as:

    - for each sample, which is to be an anchor in given generator sequence - stored in **avoids** list
    - starting indices of each classification group in table of labels/images order (each group is necessary continuous range of indices, as the table is sorted by classification label) - stored in **starts** list
    - number of images in said classification group - stored in **lengths** list
    - random starting point in fixed sequence - **seeds** list

In [ ]:
imageGroups = list(imageGroupsDct.items())
avoids, starts, lengths, seeds = [], [], [], []

# For the sake of being repeatable, fixing seed
np.random.seed(949127843)
for igrp in imageGroups:
    for i in range(igrp[1][1] - validationUniques):
        avoids.append(i+igrp[1][0] - (igrp[0]-1)*validationUniques)
        starts.append(igrp[1][0] - (igrp[0]-1)*validationUniques)
        lengths.append(igrp[1][1] - validationUniques)
        seeds.append(np.random.randint(0, high=10000000))

Updating the training data and validation data lengths. 
**validSamples** defines how many validation images are taken from the data set to make validation subset, while **validLength** defines how many validation triplets are generated from those

In [ ]:
trainLength = len(trainingTranslationTable)
validSamples = N - trainLength
validLength = len(validationSamples)

Initialising the common source of random numbers

In [ ]:
rng = tf.random.Generator.from_seed(41431)

Defining the data augmentation functions for the pipeline.

### Building the training and validation datasets pipelines

In [ ]:
get_triplet_by_index = triplet_getter(imagesTable)
augment = augmenter_getter(rng)
translate_indices = translator_getter(trainingTranslationTable)

trainDset = tf.data.Dataset.from_generator(triplet_generator,
                                    args = [starts, lengths, avoids, trainLength, seeds],
                                    output_signature=tf.TensorSpec(shape=(3), dtype=tf.int32))

trainDset = (trainDset.shuffle(trainLength, seed=6849)
      .map(translate_indices, num_parallel_calls=AUTOTUNE, deterministic=True)
      .map(get_triplet_by_index)
      .unbatch()
      .map(augment, num_parallel_calls=AUTOTUNE, deterministic=True) # augment
      .batch(3*BATCH_SIZE)
      .prefetch(2)
)

Defining the validation dataset

In [ ]:
# Cutting the validation dataset samples to fit neatly into batch sizes
validDset = (tf.data.Dataset.from_tensor_slices(validationSamples[:(len(validationSamples)//TRUE_BATCH_SIZE)*TRUE_BATCH_SIZE])
                .repeat()
                .map(get_triplet_by_index)
                .unbatch()
                .map(get_center_slice) # disabling augmentation
                .batch(3*BATCH_SIZE)
                .prefetch(2)
            )

In [ ]:
# f = validDset.take(18)
# f = iter(f)
# f = [next(f) for i in range(18)]

In [ ]:
# tf.reshape(f[0][0], (3, -1))

In [ ]:
# cov = np.cov(tf.reshape(f[0][11], (3, -1)))
# eigvals, eigvecs = np.linalg.eig(cov)

In [ ]:
# correction = tf.matmul(tf.cast(eigvecs, dtype=tf.dtypes.float32), tf.reshape(rng.normal((3,), mean=0, stddev=0.1, dtype=tf.dtypes.float32)*eigvals, (3,1)))

In [ ]:
# plt.imshow(tf.cast(f[0][11], dtype=tf.dtypes.float32)+ tf.tile(tf.reshape(correction, shape=(1,1,3)), (224,224,1)))

In [ ]:
# h = 0
# for ff in f:
#     for l in np.ravel(ff.numpy()):
#         h += hash(l)
# print(h)

In [ ]:
# fig, sub = plt.subplots(4,6)
# for i in range(4):
#     for j in range(6):
#         sub[i][j].imshow(f[0][(i*4+j)])
#         sub[i][j].axes.get_xaxis().set_visible(False)
#         sub[i][j].axes.get_yaxis().set_visible(False)

In [ ]:
# siameseBase.layers[-1].weights

# Model instantiation and training

In [ ]:
# Epochs to train
trainEpochs = 60
validationInterval = 5
trainEpochs = trainEpochs // validationInterval
previewInterval = 6*validationInterval
modelSaveInterval = 2*previewInterval

enable_validation = {
    "validation_data":validDset,
    "validation_steps":validLength//(TRUE_BATCH_SIZE), 
}
disable_validation = {
    "validation_data":None,
    "validation_steps":None, 
    "validation_batch_size":None
}

### Importing/initialising base Siamese model

Importing the  pre-trained VGG16 model with weights from imagenet without classification part. Adding 2 dense layers on top of convolutions for 4096 representation.

In [ ]:
tf.random.set_seed(1234)
# LOAD = True
# LAST_MODEL = "/home/zenfur/magister/jupyter/model_checkpoints/baseline_20210402-023106_120"
if LOAD:
    siameseBase = tf.keras.models.load_model(LAST_MODEL)
else:
    vgg16 = tf.keras.applications.VGG16(
    include_top=False,
    weights="imagenet",
    input_shape=(224,224,3),
#     pooling=None,
    )
    vgg16.trainable = False
    siameseBase = tf.keras.models.Sequential()
    for layer in (vgg16,
                    tf.keras.layers.Flatten(),
                    tf.keras.layers.Dense(4096, activation='relu'),
                    tf.keras.layers.Dense(4096, activation='relu')
                    #tf.keras.layers.Dense(4096, activation='relu')
                 ):
        siameseBase.add(layer)

In [ ]:

# siameseBase.layers[-1].kernel_regularizer = tf.keras.regularizers.l2(1e-4)
# siameseBase.layers[-1].bias_regularizer = tf.keras.regularizers.l2(1e-4)

### Compiling the triplet model

In [ ]:
tf.keras.backend.clear_session()
tripletModel = TripletSiamese(siameseBase, normalize_output_with_max, name=EXPERIMENT_NAME)
opt = tf.keras.optimizers.Adam(learning_rate=0.01)
tripletModel.compile(opt, triplet_loss)

### Preparing callbacks

In [ ]:
tensorBoardCallback = MeanTBCallback(name=EXPERIMENT_NAME,
                                 previewInterval=previewInterval,
                                 drawSamplesList=[y for x in [i[1] for i in validationSubset] for y in x],
                                 log_dir=LOG_DIR + EXPERIMENT_NAME,
                                 histogram_freq=validationInterval,
                                 profile_batch=0)


# tf.debugging.enable_check_numerics(
#     stack_height_limit=300, path_length_limit=500
# )

checkpointCallback = ModelSaverCallback(modelSaveInterval)
# monitor="loss", save_best_only=True)

In [ ]:
siameseBase.layers[0].trainable=True
for l in tripletModel.siameseBase.layers[0].layers[:-8]:
    l.trainable=False

In [ ]:
[i.trainable for i in tripletModel.siameseBase.layers[0].layers]


In [ ]:
tripletModel.siameseBase.layers[0].layers

### Executing training session

Before running be sure to check:

    - EXPERIMENT_NAME is set to distinct this run from the others
    - tensorBoardCallback uses proper log_dir
    - trainEpochs, validationInterval, previewInterval, modelSaveInterval are properly set
    - network architecture is loaded/initiated as intended
    

In [ ]:
if HIDE_WARNINGS == True:
    logging.getLogger('tensorflow').setLevel(logging.ERROR)
    
trainEpochs = 40
    
try:
    for i in range(lastEpoch, lastEpoch+trainEpochs):
        tripletModel.fit(trainDset, 
                          initial_epoch=lastEpoch,
                          epochs=lastEpoch+validationInterval-1,
                          steps_per_epoch=trainLength//(TRUE_BATCH_SIZE),
                          callbacks=[tensorBoardCallback, checkpointCallback],
                          **disable_validation
                         ) # batch_size unspecified since the batches are generated by generator
        lastEpoch += validationInterval-1
        
        tripletModel.fit(trainDset, 
                          initial_epoch=lastEpoch,
                          epochs=lastEpoch+1,
                          steps_per_epoch=trainLength//(TRUE_BATCH_SIZE),
                          callbacks=[tensorBoardCallback, checkpointCallback],
                          **enable_validation
                         )
        lastEpoch += 1

                
except KeyboardInterrupt as e:
    print("Interrupted")
logging.getLogger('tensorflow').setLevel(logging.WARNING)


In [ ]:
vgg16.trainable

In [ ]:
# plt.hist(np.ravel(siameseBase.layers[-1].weights[0]))


In [ ]:
# for i in range(6):
#     example = trainDset.take(1)
#     example = next(iter(example))
#     tripletModel.train_step(example)
#     print(i)

### Executing experiments batch

In [ ]:
experiments = [   
    {
        "name":"baselineWithDropout", # overriden if load path is specified, default == ""
        "loadPath":"", # default == ""
        "loss":"triplet_loss", # default == "triplet_loss"
        "architecture":[(4096, "relu"), (0.5, "dropout"), (4096, "relu")], # specify if not loading existing model - list of pairs N, activation function
        "vggfreeze":True,
        "batchSize":BATCH_SIZE,
        "lossParams":{"margin":1.0},
        "trainFor":240,
        "random":True,
        "optimiser":"adam",
        "optimiserParams":{"learning_rate":0.01},
        "additonalCallbacks":[], # if specified, get callback functions by name and their params
        "callbacksArguments":[]
    }, 
    {
        "name":"baselineSGD", # overriden if load path is specified, default == ""
        "architecture":[(4096, "relu"), (4096, "relu")], # specify if not loading existing model - list of pairs N, activation function
        "vggfreeze":True,
        "batchSize":BATCH_SIZE,
        "trainFor":240,
        "random":False,
        "optimiser":"adam",
        "optimiserParams":{"learning_rate":0.01},
        "outputNormalisation":"max",
    },
#     {
#         "name":"singleLayer32", # overriden if load path is specified, default == ""
#         "architecture":[(32, "relu")], # specify if not loading existing model - list of pairs N, activation function
#         "vggfreeze":True,
#         "batchSize":BATCH_SIZE,
#         "trainFor":240,
#         "random":False,
#         "optimiser":"adam",
#         "optimiserParams":{"learning_rate":0.01},
#         "outputNormalisation":"max",
#     },
#     {
#         "name":"singleLayer128", # overriden if load path is specified, default == ""
#         "architecture":[(128, "relu")], # specify if not loading existing model - list of pairs N, activation function
#         "vggfreeze":True,
#         "batchSize":BATCH_SIZE,
#         "trainFor":240,
#         "random":False,
#         "optimiser":"adam",
#         "optimiserParams":{"learning_rate":0.01},
#         "outputNormalisation":"max",
#     },
#     {
#         "name":"singleLayer256", # overriden if load path is specified, default == ""
#         "architecture":[(256, "relu")], # specify if not loading existing model - list of pairs N, activation function
#         "vggfreeze":True,
#         "batchSize":BATCH_SIZE,
#         "trainFor":240,
#         "random":False,
#         "optimiser":"adam",
#         "optimiserParams":{"learning_rate":0.01},
#         "outputNormalisation":"max",
#     },
#     {
#         "name":"singleLayerNorm512", # overriden if load path is specified, default == ""
#         "loadPath":"", # default == ""
#         "loss":"triplet_loss", # default == "triplet_loss"
#         "architecture":[(512, "relu")], # specify if not loading existing model - list of pairs N, activation function
#         "vggfreeze":True,
#         "batchSize":BATCH_SIZE,
#         "lossParams":{"margin":1.0},
#         "trainFor":240,
#         "random":False,
#         "optimiser":"adam",
#         "optimiserParams":{"learning_rate":0.01},
#         "outputNormalisation":"norm",
#         "additonalCallbacks":[], # if specified, get callback functions by name and their params
#         "callbacksArguments":[]
#     },
    {
        "name":"baselineSigmoid", # overriden if load path is specified, default == ""
        "loadPath":"", # default == ""
        "loss":"triplet_loss", # default == "triplet_loss"
        "architecture":[(4096, "relu"), (4096, "sigmoid")], # specify if not loading existing model - list of pairs N, activation function
        "vggfreeze":True,
        "batchSize":BATCH_SIZE,
        "lossParams":{"margin":1.0},
        "trainFor":240,
        "random":True,
        "optimiser":"adam",
        "optimiserParams":{"learning_rate":0.01},
        "additonalCallbacks":[], # if specified, get callback functions by name and their params
        "callbacksArguments":[]
    },
    {
        "name":"baselineSigmoidNoNorm", # overriden if load path is specified, default == ""
        "loadPath":"", # default == ""
        "loss":"triplet_loss", # default == "triplet_loss"
        "architecture":[(4096, "relu"), (4096, "sigmoid")], # specify if not loading existing model - list of pairs N, activation function
        "vggfreeze":True,
        "batchSize":BATCH_SIZE,
        "lossParams":{"margin":1.0},
        "outputNormalisation":None,
        "trainFor":240,
        "random":True,
        "optimiser":"adam",
        "optimiserParams":{"learning_rate":0.01},
        "additonalCallbacks":[], # if specified, get callback functions by name and their params
        "callbacksArguments":[]
    },
    {
        "name":"threeLayerThen512Dropout", # overriden if load path is specified, default == ""
        "loadPath":"", # default == ""
        "loss":"triplet_loss", # default == "triplet_loss"
        "architecture":[(4096, "relu"), (0.5, "dropout"), (4096, "relu"), (0.5, "dropout"), (512, "relu"),], # specify if not loading existing model - list of pairs N, activation function
        "vggfreeze":True,
        "batchSize":BATCH_SIZE,
        "lossParams":{"margin":1.0},
        "trainFor":240,
        "random":True,
        "optimiser":"adam",
        "optimiserParams":{"learning_rate":0.01},
        "additonalCallbacks":[], # if specified, get callback functions by name and their params
        "callbacksArguments":[]
    },
    {
        "name":"baseline", # overriden if load path is specified, default == ""
        "loadPath":"", # default == ""
        "loss":"triplet_loss", # default == "triplet_loss"
        "architecture":[(4096, "relu"), (4096, "relu")], # specify if not loading existing model - list of pairs N, activation function
        "vggfreeze":True,
        "batchSize":BATCH_SIZE,
        "lossParams":{"margin":1.0},
        "trainFor":240,
        "random":True,
        "optimiser":"adam",
        "optimiserParams":{"learning_rate":0.01},
        "additonalCallbacks":[], # if specified, get callback functions by name and their params
        "callbacksArguments":[]
    },
        {
        "name":"baseline", # overriden if load path is specified, default == ""
        "loadPath":"", # default == ""
        "loss":"triplet_loss", # default == "triplet_loss"
        "architecture":[(4096, "relu"), (4096, "relu")], # specify if not loading existing model - list of pairs N, activation function
        "vggfreeze":True,
        "batchSize":BATCH_SIZE,
        "lossParams":{"margin":1.0},
        "trainFor":240,
        "random":True,
        "optimiser":"adam",
        "optimiserParams":{"learning_rate":0.01},
        "additonalCallbacks":[], # if specified, get callback functions by name and their params
        "callbacksArguments":[]
    },
#     {
#         "name":"baselineLong", # overriden if load path is specified, default == ""
#         "loadPath":"", # default == ""
#         "loss":"triplet_loss", # default == "triplet_loss"
#         "architecture":[(4096, "relu"), (4096, "relu")], # specify if not loading existing model - list of pairs N, activation function
#         "vggfreeze":True,
#         "batchSize":BATCH_SIZE,
#         "lossParams":{"margin":1.0},
#         "trainFor":900,
#         "random":False,
#         "optimiser":"adam",
#         "optimiserParams":{"learning_rate":0.01},
#         "additonalCallbacks":[], # if specified, get callback functions by name and their params
#         "callbacksArguments":[]
#     },
#     {
#         "name":"wholeNet", # overriden if load path is specified, default == ""
#         "loadPath":"", # default == ""
#         "loss":"triplet_loss", # default == "triplet_loss"
#         "architecture":[(4096, "relu"), (4096, "relu")], # specify if not loading existing model - list of pairs N, activation function
#         "vggfreeze":False,
#         "batchSize":4,
#         "lossParams":{"margin":1.0},
#         "trainFor":240,
#         "random":False,
#         "optimiser":"adam",
#         "optimiserParams":{"learning_rate":0.01},
#         "additonalCallbacks":[], # if specified, get callback functions by name and their params
#         "callbacksArguments":[]
#     },
    {
        "name":"wholeNetSGDhlr", # overriden if load path is specified, default == ""
        "loadPath":"", # default == ""
        "loss":"triplet_loss", # default == "triplet_loss"
        "L2penalty":5e-4,
        "architecture":[(4096, "relu"), (4096, "relu")], # specify if not loading existing model - list of pairs N, activation function
        "vggfreeze":False,
        "batchSize":4,
        "lossParams":{"margin":1.0},
        "trainFor":60,
        "random":False,
        "optimiser":"sgd",
        "optimiserParams":{"learning_rate":0.05, "momentum":0.9},
        "additonalCallbacks":[], # if specified, get callback functions by name and their params
        "callbacksArguments":[]
    },
    {
        "name":"wholeNetSGDllr", # overriden if load path is specified, default == ""
        "loadPath":"", # default == ""
        "loss":"triplet_loss", # default == "triplet_loss"
        "L2penalty":5e-4,
        "architecture":[(4096, "relu"), (4096, "relu")], # specify if not loading existing model - list of pairs N, activation function
        "vggfreeze":False,
        "batchSize":4,
        "lossParams":{"margin":1.0},
        "trainFor":60,
        "random":False,
        "optimiser":"sgd",
        "optimiserParams":{"learning_rate":0.001, "momentum":0.9},
        "additonalCallbacks":[], # if specified, get callback functions by name and their params
        "callbacksArguments":[]
    },
        {
        "name":"baseline", # overriden if load path is specified, default == ""
        "loadPath":"", # default == ""
        "loss":"triplet_loss", # default == "triplet_loss"
        "architecture":[(4096, "relu"), (4096, "relu")], # specify if not loading existing model - list of pairs N, activation function
        "vggfreeze":True,
        "batchSize":BATCH_SIZE,
        "lossParams":{"margin":1.0},
        "trainFor":240,
        "random":True,
        "optimiser":"adam",
        "optimiserParams":{"learning_rate":0.01},
        "additonalCallbacks":[], # if specified, get callback functions by name and their params
        "callbacksArguments":[]
    },
        {
        "name":"baseline", # overriden if load path is specified, default == ""
        "loadPath":"", # default == ""
        "loss":"triplet_loss", # default == "triplet_loss"
        "architecture":[(4096, "relu"), (4096, "relu")], # specify if not loading existing model - list of pairs N, activation function
        "vggfreeze":True,
        "batchSize":BATCH_SIZE,
        "lossParams":{"margin":1.0},
        "trainFor":240,
        "random":True,
        "optimiser":"adam",
        "optimiserParams":{"learning_rate":0.01},
        "additonalCallbacks":[], # if specified, get callback functions by name and their params
        "callbacksArguments":[]
    },
        {
        "name":"baseline", # overriden if load path is specified, default == ""
        "loadPath":"", # default == ""
        "loss":"triplet_loss", # default == "triplet_loss"
        "architecture":[(4096, "relu"), (4096, "relu")], # specify if not loading existing model - list of pairs N, activation function
        "vggfreeze":True,
        "batchSize":BATCH_SIZE,
        "lossParams":{"margin":1.0},
        "trainFor":240,
        "random":True,
        "optimiser":"adam",
        "optimiserParams":{"learning_rate":0.01},
        "additonalCallbacks":[], # if specified, get callback functions by name and their params
        "callbacksArguments":[]
    },        {
        "name":"baseline", # overriden if load path is specified, default == ""
        "loadPath":"", # default == ""
        "loss":"triplet_loss", # default == "triplet_loss"
        "architecture":[(4096, "relu"), (4096, "relu")], # specify if not loading existing model - list of pairs N, activation function
        "vggfreeze":True,
        "batchSize":BATCH_SIZE,
        "lossParams":{"margin":1.0},
        "trainFor":240,
        "random":True,
        "optimiser":"adam",
        "optimiserParams":{"learning_rate":0.01},
        "additonalCallbacks":[], # if specified, get callback functions by name and their params
        "callbacksArguments":[]
    },
        {
        "name":"baseline", # overriden if load path is specified, default == ""
        "loadPath":"", # default == ""
        "loss":"triplet_loss", # default == "triplet_loss"
        "architecture":[(4096, "relu"), (4096, "relu")], # specify if not loading existing model - list of pairs N, activation function
        "vggfreeze":True,
        "batchSize":BATCH_SIZE,
        "lossParams":{"margin":1.0},
        "trainFor":240,
        "random":True,
        "optimiser":"adam",
        "optimiserParams":{"learning_rate":0.01},
        "additonalCallbacks":[], # if specified, get callback functions by name and their params
        "callbacksArguments":[]
    },
]
# add unfreezing experiments

In [ ]:
experimentDefaults =     {
        "name":"baseline", # overriden if load path is specified, default == ""
        "loadPath":"", # default == ""
        "loss":"triplet_loss", # default == "triplet_loss"
        "architecture":[(4096, "relu"), (4096, "relu")], # specify if not loading existing model - list of pairs N, activation function
        "vggfreeze":True,
        "batchSize":4,
        "lossParams":{"margin":1.0},
        "trainFor":360,
        "validationInterval":5,
        "random":False,
        "randomDataOrder":False,
        "optimiser":"adam",
        "optimiserParams":{"learning_rate":0.01},
        "outputNormalisation":"max",
        "additonalCallbacks":None, # if specified, get callback functions by name and their params
        "callbacksArguments":None,
        "L2penalty":0
}
# TODO: implement additional callbacks and arguments


In [ ]:
for experiment in experiments:
    for key, value in experimentDefaults.items():
        if key not in experiment:
            experiment[key] = value # modifying the original
    try:
        # Parse load model if path specified
        if "loadPath" in experiment and experiment["loadPath"]:
            baseName = os.path.basename(experiment["loadPath"])
            s = baseName.split('_')
            # read last epoch from model name
            lastEpoch = int(s[-1]) + 1
            # set previous experiment name
            EXPERIMENT_NAME = "_".join(s[:-1])
            del s
            siameseBase = tf.keras.models.load_model(experiment["loadPath"])
        else:
        # Model path not specified - has to be initialised
            set_experiment_name(experiment["name"])
            vgg16 = tf.keras.applications.VGG16(
                include_top=False,
                weights="imagenet",
                input_shape=(224,224,3),
            )
            lastEpoch=1
            
            if experiment["random"] == False:
                tf.random.set_seed(11123)
                np.random.seed(1112332)
            else:
                seed = int((datetime.now()- datetime(1970,1,1)).total_seconds())
                np.random.seed(seed//2)
                tf.random.set_seed(seed)
            shuffleSeed = tf.random.get_global_generator().uniform_full_int([1]).numpy()[0]

            
            # Instantiate the model
            vgg16.trainable = not experiment.get("vggfreeze", True)
            siameseBase = tf.keras.models.Sequential()
            layers = [vgg16, tf.keras.layers.Flatten()]
            for layerArch in experiment["architecture"]:
                if layerArch[1] == "dropout":
                    layers.append(tf.keras.layers.Dropout(layerArch[0]))
                else:
                    layers.append(tf.keras.layers.Dense(layerArch[0], layerArch[1]))
            for layer in layers:
                siameseBase.add(layer)
            
            if experiment["L2penalty"] > 0:
                for l in siameseBase.layers[0].layers:
                    if (isinstance(l, tf.keras.layers.Dense) or isinstance(l, tf.keras.layers.Conv2D)) and l.trainable:
                        l.kernel_regularizer = tf.keras.regularizers.l2(experiment["L2penalty"])
                        l.bias_regularizer = tf.keras.regularizers.l2(experiment["L2penalty"])
                for l in siameseBase.layers:
                    if (isinstance(l, tf.keras.layers.Dense) or isinstance(l, tf.keras.layers.Conv2D)) and l.trainable:
                        l.kernel_regularizer = tf.keras.regularizers.l2(experiment["L2penalty"])
                        l.bias_regularizer = tf.keras.regularizers.l2(experiment["L2penalty"])
        
        # Compile the model
        if not experiment["outputNormalisation"]:
            normalisation_fn = lambda x:x
        elif experiment["outputNormalisation"] == "max":
            normalisation_fn = normalize_output_with_max
        elif experiment["outputNormalisation"] == "batch":
            normalisation_fn = normalize_output_with_max_batch
        elif experiment["outputNormalisation"] == "norm":
            normalisation_fn = normalize_output_with_norm

        tripletModel = TripletSiamese(siameseBase, normalisation_fn, name=EXPERIMENT_NAME)
        # TODO - choose optimizer according to settings
        
        if experiment["optimiser"] == "adam":
            opt = tf.keras.optimizers.Adam(**experiment["optimiserParams"])
        elif experiment["optimiser"] == "sgd":
            opt = tf.keras.optimizers.SGD(**experiment["optimiserParams"])
        # TODO - choose loss according to  settings
        # TODO - use loss params
        tripletModel.compile(opt, triplet_loss)

        
        # (Re)Initialise dataset
        batchSize = experiment["batchSize"] or BATCH_SIZE
        # TODO - roll generator to match its current epoch if loading
        rng = tf.random.Generator.from_seed(41431)
        
        if experiment["randomDataOrder"]:
            newSeeds = []
            for igrp in imageGroups:
                for i in range(igrp[1][1] - validationUniques):
                    newSeeds.append(np.random.randint(0, high=10000000))
        else: # only relevant to loading model
            newSeeds = advance_triplet_generator((lastEpoch-1)*trainLength, lengths, trainLength, seeds)
        
        trainDset = tf.data.Dataset.from_generator(triplet_generator,
                                            args = [starts, lengths, avoids, trainLength, newSeeds],
                                            output_signature=tf.TensorSpec(shape=(3), dtype=tf.int32)) 
        get_triplet_by_index = triplet_getter(imagesTable)
        augment = augmenter_getter(rng)
        translate_indices = translator_getter(trainingTranslationTable)

        trainDset = (trainDset.shuffle(trainLength, seed=shuffleSeed)
              .map(translate_indices, num_parallel_calls=AUTOTUNE, deterministic=True)
              .map(get_triplet_by_index)
              .unbatch()
              .map(augment, num_parallel_calls=AUTOTUNE, deterministic=True)
              .batch(3*batchSize)
              .prefetch(2)
        )
        
        
        tensorBoardCallback = MeanTBCallback(name=EXPERIMENT_NAME,
                                 previewInterval=previewInterval,
                                 drawSamplesList=[y for x in [i[1] for i in validationSubset] for y in x],
                                 log_dir=LOG_DIR + EXPERIMENT_NAME,
                                 histogram_freq=validationInterval,
                                 profile_batch=0)

        checkpointCallback = ModelSaverCallback(modelSaveInterval)
        
        # TODO - handle custom callbacks
        
        if HIDE_WARNINGS == True:
            logging.getLogger('tensorflow').setLevel(logging.ERROR)
        
        trainEpochs = experiment["trainFor"]
        validationInterval = experiment["validationInterval"]
        trainEpochs = (trainEpochs+validationInterval-1) // validationInterval
        for i in range(lastEpoch, lastEpoch+trainEpochs):
            tf.print(EXPERIMENT_NAME)
            tripletModel.fit(trainDset, 
                              initial_epoch=lastEpoch,
                              epochs=lastEpoch+validationInterval-1,
                              steps_per_epoch=trainLength//(batchSize*3),
                              callbacks=[tensorBoardCallback, checkpointCallback],
                              **disable_validation
                             )
            lastEpoch += validationInterval-1

            tripletModel.fit(trainDset, 
                              initial_epoch=lastEpoch,
                              epochs=lastEpoch+1,
                              steps_per_epoch=trainLength//(batchSize*3),
                              callbacks=[tensorBoardCallback, checkpointCallback],
                              **enable_validation
                             )
            lastEpoch += 1

#     except KeyboardInterrupt as e:
#         print("Interrupted")
#     logging.getLogger('tensorflow').setLevel(logging.WARNING)
        tf.keras.backend.clear_session()
    
    
    except Exception as e:
        tf.print(f"Skipping experiment {EXPERIMENT_NAME} due to exception ", e)

# Post training analysis

### Tensorboard plugin

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir $LOG_DIR

In [ ]:
plotdirs = ["/home/zenfur/magister/jupyter/siamese_logs/baseline_20210424-044333/validation", 
           "/home/zenfur/magister/jupyter/siamese_logs/baseline_20210424-044333/train"]

In [ ]:
dir_ = plotdirs[0]
eventfiles = [dir_+"/"+x for x in os.listdir(dir_)]

In [ ]:
# os.listdir("/home/zenfur/magister/jupyter/siamese_logs/")
root, dirs, _ = next(os.walk(LOG_DIR))
dirs = [root + d for d in dirs]

In [ ]:
pastExperiments = list()
for d in dirs:
    subdirs = os.listdir(d)
    if "validation" in subdirs:
        for sd in subdirs:
            pastExperiments.append(d + "/" + sd)
    else:
        print(f"Omitting {d} experiment - no validation data")

In [ ]:
chronoOrder = np.argsort([e.split('/')[-2].split('_')[1] for e in pastExperiments])

with open("experiments.list", "w") as file:
    writer = csv.writer(file)
    for chord in chronoOrder:
        splits = pastExperiments[chord].split('/')
        name, date = splits[-2].split('_')
        comment = ""
        writer.writerow((date, name, pastExperiments[chord], comment))

In [ ]:
def merge_tbevents(dirpath, tag):
    eventfiles = [dirpath+"/"+x for x in os.listdir(dirpath)]
    # Assumption - maximum of data entry per file
    xx = np.zeros(len(eventfiles))
    yy = np.zeros(len(eventfiles))
    for n, evfile in enumerate(eventfiles):
        itr = tf.compat.v1.train.summary_iterator(evfile)
        for i in itr:
            step = i.step
            if i.summary and i.summary.value:
                for v in i.summary.value:
                    if v.tag == tag:
                        f = struct.unpack('f', v.tensor.tensor_content)
                        xx[n] = step
                        yy[n] = f[0]
                        
    order = np.argsort(xx)
    xx = xx[order]
    yy = yy[order]
    return xx,yy

def discover_tags(dirpath):
    eventfiles = [dirpath+"/"+x for x in os.listdir(dirpath)]
    tags = set()
    for n, evfile in enumerate(eventfiles):
        itr = tf.compat.v1.train.summary_iterator(evfile)
        for i in itr:
            step = i.step
            if i.summary and i.summary.value:
                for v in i.summary.value:
                    tags.add(v.tag)
    return tags

In [ ]:
discover_tags(plotdirs[1])

In [ ]:
xx,yy = merge_tbevents(plotdirs[0], "mean_loss")

In [ ]:
plt.plot(xx,yy)

For all last saved models calculating the analysis plots like PCA if they are missing

In [ ]:
models = os.listdir(MODEL_DIR)
modelsDict = {}
# keep only last model from each model sequence
for model in models:
    splits = model.split('_')
    try:
        epoch = int(splits[-1])
    except ValueError as e:
        continue
    if epoch == 240:
        baseName = "_".join(splits[:-1])
        if modelsDict.get(baseName, 0) < epoch:
            modelsDict[baseName] = epoch
        
modelsToAnalyse = [f"{MODEL_DIR}{k}_{v}" for k,v in modelsDict.items()]

In [ ]:
modelName = "baseline_20210424-044333_240"

In [ ]:
modelsToAnalyse

In [ ]:
# disable warnings
logging.getLogger('tensorflow').setLevel(logging.ERROR)

for modelName in progressBar(modelsToAnalyse):
    tf.keras.backend.clear_session()
    name = modelName.split('/')[-1]
    pcaPlotName = f"{ANALYSIS_DIR}{name}_pca.png"
    if not os.path.exists(pcaPlotName):
        model = tf.keras.models.load_model(modelName)
        rep = model.predict(tf.image.central_crop(imagesTable, 224/256))
        pcaVals, pcaVec = getPCA(abs(rep))
        f, p = plt.subplots(1,1)
        p.semilogy(np.sort(abs(pcaVals))[::-1])
        p.set_title(f"PCA values output of {name}")
        f.savefig(fname=pcaPlotName)
    del model
    
# enable warnings
logging.getLogger('tensorflow').setLevel(logging.WARNING)

In [ ]:
tf.norm(tf.constant([[0,0,0,0], [0,0,0,0]], dtype=tf.dtypes.float32), axis=1)

In [ ]:

x1 = tf.constant([[0,0,0,0], [1.0,0,0,0]], dtype=tf.dtypes.float32)
x2 = tf.constant([[0,0,0,0], [0,0,0,0]], dtype=tf.dtypes.float32)
with tf.GradientTape() as tape:
    tape.watch(x1)
    tape.watch(x2)
    #l = tf.norm(x2-x1, axis=1)
    l = tf.reduce_sum(tf.math.squared_difference(x2, x1), axis=1)
    #b[b==0.0] = tf.constant(float("inf"))
print(tape.gradient(l, x1))